In [1]:
from docx import *
import pandas as pd
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [2]:
# read file
document = Document("Preprocessing/[3]Dictionary IND-ENG-208-230.docx")
# document = Document("Converted/pdf2go-execution.docx")

In [3]:
for paragraph in document.paragraphs:
    
#     print(len(paragraph.runs))
    
#     for run in paragraph.runs :
#         print(run.text)
    
    
    # first line indent (segitiga atas di ruler)
    first = paragraph.paragraph_format.first_line_indent
    
    # left indent (segitiga bawah di ruler)
    left = paragraph.paragraph_format.left_indent
    
    print(first,left,paragraph.text+"\n")

None 76200 cokin (Pr) (a slur for) Indonesian<born Chinese; > CINA.

-152400 227965 coklat (D) 1 pohon – cacao tree, Theobroma cacao, whose seeds are made into a powder.

-152400 227965 2 chocolate, a drink made by adding sugar and milk/hot water to the above powder.

-152400 227965 3 chocolate, sweet substance made with cocoa powder, sugar, etc.

-152400 227965 4 chocolate, a reddish<yellow brown color.

-152400 227965 Gadis itu mengenakan seragam – dengan pét putih.

-152400 227965 The girl was wearing a reddish<yellow uni< form with white cap.

-152400 227965 – betas sugared caraway seeds (to be sprinkled on bread).

-152400 227965 – sampang rosewood (in color).

-152400 227965 – susu milk chocolate.

-152400 227965 – tembakau snuff brown.

None 152400 secoklat as brown as.

None 152400 mencoklat to become brown.

None 152400 mencoklati and mencoklatkan to make something brown, darken.

-76200 228600 kecoklatan cacao (mod).

-76200 228600 Tingkat ~nya sempurna dan aromanya pun bagus

-153035 228600 Tiangnya – sedikit.

-153035 228600 The pole is somewhat slanting.

-153035 228600 2 slightly crazy.

-153035 228600 pikiran – crazy.

-153035 228600 – ingatan slightly crazy.

-153035 228600 – mondong all aslant (of pillars, trees).

-153035 228600 3 to set, sink, go down (of the sun).

-153035 228600 Matahari – ke barat.

-153035 228600 The sun sets in the west.

-153035 228600 hatinya – (ke)pada to be inclined to one party, in favor of, take someone’s side.

-153035 228600 4 (M) to trim one’s sails to every wind.

-153035 228600 5 tendency.

-76200 228600 mencondongkan 1 to bend, lean.

-76200 228600 ~ badan ke depan to bend/lean forward, stoop.

-76200 228600 2 to incline, tend.

-76200 228600 ~ kepada to favor, 3 to show affection for.

None 152400 tercondong inclined.

None 152400 ~ balik reclined.

None 152400 kecondongan 1 tendency, penchant.

None 152400 2 inclination, leaning.

None 152400 3 bias.

None 76200 condrong > CENDERUNG.

-152400 None Conéfo /ko</ [Co

None 227965 Achmad Tirtosudiro.

None 227965 2 foundry.

None 227965 ~ beton con< crete mixer.

None 227965 ~ cétak die casting.

None 227965 ~ terakhir topping up (of a building under construction).

-152400 227965 corak 1 (fabric) design, motif, pattern.

-152400 227965 2 stripe (on a ¶ag).

-152400 227965 3 color.

-152400 227965 – politiknya tidak tegas.

-152400 227965 His political color is unclear.

-152400 227965 4 type, form, structure, feature, characteristic.

-152400 227965 – pikiran attitude.

-152400 227965 – ragam sorts, kinds, types.

-152400 227965 bercorak ragam manifold, multifarious.

-76200 227965 bercorak 1 with/to have a pattern, patterned.

-76200 227965 2 with stripes, striped.

-76200 227965 3 with a color, tinged, characterized.

-76200 227965 informasi ~ kebu- dayaan yang mempunyai tugas mendidik cultural<educative information.

None 151765 mencoraki to model.

None 75565 corak-carik tattered, in tatters/rags.

None 151765 mencorak-carikkan to tear something

-152400 None – crat a) repeated acts as above.

-152400 None b) to keep on going to the bathroom.

None 152400 crét-crét (onom) squirt, sound of squirting/spraying.

None 152400 Tiba-tiba,

None None 

None None 

None 0 

None 228600 “Crét, crét!” Seseorang menyemprotkan sesuatu ke arah mata Sapto.

None 228600 Suddenly, squirt, squirt.

None 228600 Somebody sprayed something toward Sapto’s eyes.

None 152400 ngecrét 1 to squirt.

None 152400 2 to ejaculate, come.

None 75565 crigis (Jv) to nag, bother.

None 75565 gadis yang – a nagging girl.

None 75565 cring (onom) jingling sound.

-153035 227965 criping and crimping (Jv) fried chips (made from cassava, etc.).

-153035 227965 – kentang potato chips.

-153035 227965 – ketéla cassava chips.

-153035 227965 – pisang fried ba< nana chips.

None 75565 crit > CRÉT.

None 75565 crocos (Jv) pating – ( plural) ¶owing in profusion; > CEROCOS.

-76200 228600 nyrocos (coq) to ¶ow/pour out.

-76200 228600 ~lah info dari mulut perjaka itu.

-762

-76200 227965 mencuci [and nyuci (coq)] 1 to wash, cleanse.

-76200 227965 ~ kendaraan to wash a car.

-76200 227965 ~ léhér to eat well, have a good meal.

-76200 227965 ~ mata to look around, sightsee.

-76200 227965 ~ tangan a) to wash one’s hands (with water).

None 227965 b) to attempt to show that one is guiltless.

None 227965 dapat dicuci dengan tangan hand<washable.

None 227965 dicuci kering dry<cleaned.

None 227965 Dan bila ada waktu baru dibawa ke binatu yang profésional untuk dicuci ke- ring and if there’s time it’s taken to the laundry to be dry<

None None 

None 0 

None None 

None 227965 cleaned.

None 227965 2 to develop (¤lm).

None 227965 dan ~ dan mencétak ¤lm warna to develop and print color ¤lm.

None 227965 3 to wash, launder, do the laundry.

None 227965 ~ pakaian to wash the clothes.

None 227965 ~ uang to launder money.

None 227965 4 to purge (the stomach), purify (the blood).

None 151765 mencucikan [and nyuciin (J coq)] 1 to wash something for someone 2 

...

None 151765 perdapuran kitchen (mod), culinary.

-153035 228600 dapur II dapur-dapur the external part of an object.

-153035 228600 ~ bahu the part of the body under the shoulder, thorax.

-153035 228600 ~ susu the part of the body under the breasts.

-152400 227965 dar I (A) (in compounds) 1 house surrounding a yard; dwelling, res< idence.

-152400 227965 2 yard.

-152400 227965 – al<aitam and – ul<aitam orphanage.

-152400 227965 – ul<akhirat the hereafter.

-152400 227965 – ul<Arqam the house of Arqam, named after a friend of the Prophet who donated his house for Islamic mission< ary activities.

-152400 227965 – ul<asyikin the world of passions.

-152400 227965 – ul<atsar mu< seum.–ul<bakathehereafter.–fanathetransitoryworld.–ul<harb the non<Muslim world.

-152400 227965 – ul<ilmu university.

-152400 227965 – ul<iqab hell.

-152400 227965 – ul< Islam a) the Muslim faith.

-152400 227965 b) [DI] a rebel movement in southern Celebes in the 1950s.

-152400 227965 – ul<jalal heaven.


-153035 228600 – kupu< kupu bow tie.

-153035 228600 – panjang necktie.

-153035 228600 – péndék short necktie.

None 152400 berdasi with a necktie; to wear a necktie.

None 76200 dastar and daster I > DESTAR.

None 13970 daster II (E) duster, housecoat, lab coat.

None 13970 berdaster to wear a duster/lab coat.

None 13970 dastur (Pers naut) layar – studding<sail.

-152400 228600 dasun (M) garlic, kind(s) of Allium.

-152400 228600 – tunggal a single/solitary garlic (not a dual or triple one).

-152400 228600 hidung bak/seperti – tunggal a well<shaped nose: round at the end and sharp at the back.

-152400 228600 hidung mendasun tunggal to have such a nose.

-152400 None Daswati [Daérah Swatantra Tingkat I] First Level Autonomous District.

None 76200 dat > ZAT.

-153035 228600 data (E) data, information.

-153035 228600 sistém basis – database.

-153035 228600 kedap – data proof.

-153035 228600 – dasar/pokok basic data, database.

-153035 228600 – keluar output.

-153035 228600 – mas

None 76200 datia (Skr) > DAITIA.

None 76200 datif (D gram) dative.

-152400 228600 datin (Mal) 1 title given to the wives of datuk.

-152400 228600 2 honorary title for female (equivalent to datuk).

None 76200 dating (E) 1 date.

None 76200 2 dating.

None 76200 Dato’ > DATUK.

-152400 None datu I 1 (ob) sovereign, monarch, king; > DATUK, RATU.

-152400 None 2 (M) med< icine man, shaman; > BOMOH, DUKUN.

-152400 None 3 (J) a holy person, saint; a deceased person.

-76200 228600 kedatuan (ob) 1 kingdom.

-76200 228600 ~ Sriwijaya the Sriwijaya Kingdom (of Sumatra).

None 76200 datu II kedatuan (M) kind(s) of dance.

-152400 None datuk 1 head, elder.

-152400 None 2 grandfather.

-152400 None 3 (M) an honori¤c title, * a gelar for people of high status or older people within the group.

-152400 None 4 idol (in Chinese temple); > TOA-PÉKONG.

-152400 None 5 (taboo term for) ti< ger.

-152400 None si – reference to the Sumatran elephant.

-152400 None – bandar (Mal) mayor; > WALIKOTA(MAD

0 None – putri kind(s) of plant, Mussaenda spp.

0 None – rahat part of spin< ning wheel for winding thread.

0 None – rebah bangun mimosa, Mimosa pudica.

0 None – rokok cigarette paper.

0 None – rusa kind(s) of plant, scarlet hygro, sessile joyweed, Alternanthera sessilis.

0 None – salam (Jv) bay leaf.

0 None – sambung nyawa kind(s) of tree, Gynura procumbens.

0 None – sang joey palm, Johannesteijsmannia altifrons.

0 None – sangkétan prickly chaff ¶ower, devil’s horsewhip, Achyranthes aspera.

0 None – saputangan a decora< tive plant, Maniltoa grandi¶ora/schefferi.

0 None – séndok kind(s) of plantain,

None 227965 
Plantago major.

None 227965 – simbukan kind(s) of shrub, Saprosma ternatum.

None 227965 – sudamala mugwort, Artemisia vulgaris.

None 227965 – tampar hantu kind(s) of plant.

None 227965 – telinga auricle, outer edge of the ear.

None 227965 – tempuyung sow thistle, Sonchus arvenis.

None 227965 – tembaga ¶ame violet, Episcia cupreata.

-635 228600 ténda (bio) tepa

None 125730 daya III > BARAT daya.

None 125730 dayah I (Pers) 1 wet nurse.

None 125730 2 governess.

None 125730 dayah II (Ach) Islamic boarding school; > PESANTRÉN.

None 125730 Daya(k) Dayak.

None 125730 dayak (BG) unwelcome guest, gate<crasher.

None 125730 dayan (A) ad – a name of God, the Reciprocator.

-152400 278765 dayang (cla) 1 girl.

-152400 278765 2 lady<in<waiting.

-152400 278765 3 (modern meaning) chap< erone.

None 202565 dayang-dayang (cla) lady<in<waiting, handmaid.

None 202565 mendayangi to serve (someone in court).

None 202565 pendayang 1 (cla) young lady.

None 202565 2 prostitute.

None 202565 pendayangan female attendants in the palace.

None 202565 perdayangan seraglio, female court servants.

None 126365 dayu I 1 (onom) moaning, groaning, rumble.

None 126365 2 sighing, plaint.

None 202565 mendayu 1 to moan, groan.

None 202565 2 to rumble (of distant thunder).

None 202565 3

None 278765 to lull (to sleep).

None 202565 mendayu-dayu to lull to sleep.

No

In [4]:
# Target yang di crawling
# [lema, sublema, gabungan(-,~), similar(panah,"and"), keterangan, akronim([]), terjemahan]

# CARA NGEDAPETIN (bantu isi cara ngedapetin atau ciri khas atau langkah2 buat ngedapetin datanya)
# [v] == sudah dibuat codingannya

# [v] LEMA : indent pertama,bold (simpan di tempLema)
# [v] LEMA (jika sebaris dg lema lainnya) : indent pertama, jika kondisi tempLema isi, jika tidak bisa distemming dan huruf awal kata sama seperti awal kata tempLema
# [v] LEMA (jika sebaris dg sublema) : indent kedua, jika kondisi tempSublema harus isi, jika tidak bisa distemming dan huruf awal kata sama seperti awal kata tempSublema

# [v] SUBLEMA : indent kedua,deteksi bold (simpan di tempSublema)
# [v] SUBLEMA (jika sebaris dg lema): indent pertama, jika kondisi tempLema isi, jika bisa distemming dan huruf awal kata tidak sama seperti awal kata tempLema
# [v] SUBLEMA (jika sebaris dg sublema lainnya) : indent kedua, jika kondisi tempSublema harus isi, jika bisa distemming dan huruf awal kata tidak sama seperti awal kata tempSublema

# [v] GABUNGAN(-,~) : deteksi kata italic, ambil kata-katanya simpan ke list tempGabungan, cek di list ada - atau ~ tidak, kalau ada berarti itu kata gabungan dan replace - dan ~ dengan lema/sublema 

# [v] SIMILAR ("and") : indent pertama/kedua, deteksi bold, jika kata sebelumnya adalah and maka similar
# SIMILAR (panah) :

# [v] KETERANGAN :

# [v] AKRONIM :

# TERJEMAHAN :

In [5]:
# Perisiapan Word :
# - ubah panah jadi >
# - –/~ yang reguler dan bold dijadikan italic
# - ubah font, kemudian tanda yang tidak terbaca seperti di kata "major" (-^w) diganti menjadi <
# - tanda yang tidak terbaca seperti di kata "brother-in-law" (-) diganti menjadi ^w
# - tanda "- " diganti menjadi "<", contoh me- ngelabui (PERTIMBANGKAN LAGI)
# - ubah tanda "..." menjadi "_" (garis bawah)
# - s.t. = something
# - s.o.'s = someone's
# - s.o. = someone
# - q.v. = which see (refer to that word)
# - k.o. = kind(s) of
# - o.s. = oneself
# - e.o. = eachother
# - replace .^w jadi .^p
# - menghapus setiap header halaman dan garisnya



In [6]:
def cleanWord(word) :
    #cek spasi pada kata (jika terdiri dari 2 kata)
    if re.search("\s", word) :
        word = word.split()

#         word[:] = [w for w in word if not (w in roman_numerals or re.search("\d", w) or word == "/" or w == "." or w == "(")]
        
        #update 1
        word[:] = [w for w in word if not (w in roman_numerals or re.search("\d", w))]
    
        word = " ".join(word)
        
        if len(word) < 1 :
            word = None

    #jika terdiri dari 1 kata
    else :
        #update 3
#         if word in roman_numerals or re.search("\d", word) or word == "/" or word == "." or word == "(":
        
        if word in roman_numerals or re.search("\d", word) :
            word = None
    
    return(word)

def replaceData (x) :
    for idx in range(len(x)):
        x[idx] = re.sub("¤","fi",x[idx])
        x[idx] = re.sub("¶","fl",x[idx])
        x[idx] = re.sub("\s{2}|\s{3}"," ",x[idx])
        x[idx] = re.sub("\s*$|\xad|\xad\s*","",x[idx])
        x[idx] = re.sub("^\)|^\)\s*","",x[idx])
        x[idx] = re.sub("\(\s\)","",x[idx])
        x[idx] = re.sub("^\w\)\s*|\s\w\)$","",x[idx])
        x[idx] = re.sub("^\s*|^\s|^\s\w","",x[idx])
        x[idx] = re.sub("\[and","",x[idx])
        x[idx] = re.sub("\[|\[\s","",x[idx])
        x[idx] = re.sub("\]","",x[idx])
        x[idx] = re.sub("\s-\s|-\s","-",x[idx])
        x[idx] = re.sub("\s<\s|\s<|<\s|<","",x[idx])
        x[idx] = re.sub("^\s","",x[idx])
        x[idx] = re.sub("\s\)",")",x[idx])
        x[idx] = re.sub("\(\s","(",x[idx])
        x[idx] = re.sub("^/.*/","",x[idx])
        x[idx] = re.sub("/\s","/",x[idx])
        x[idx] = re.sub("\snya","nya",x[idx])
        x[idx] = re.sub("\skan$","kan",x[idx])
        x[idx] = re.sub("\skan\s","kan ",x[idx])
        
#         x[idx] = re.sub("s\.t\.|s\.t","something",x[idx])
#         x[idx] = re.sub("s\.o\.'s","someone's",x[idx])
#         x[idx] = re.sub("s\.o\.","someone",x[idx])
#         x[idx] = re.sub("q\.v\.","which see (refer to that word)",x[idx])
#         x[idx] = re.sub("k\.o\.","kind(s) of",x[idx])
#         x[idx] = re.sub("o\.s\.","oneself",x[idx])
#         x[idx] = re.sub("e\.o\.","eachother",x[idx])
#         x[idx] = re.sub(";|;\s","",x[idx])

        x[idx] = re.sub("\*\*","e.g.",x[idx])
        x[idx] = re.sub("\*","i.e.,",x[idx])
        x[idx] = re.sub("\.$|\s\.$|\.\s$|\.\s","",x[idx])
#         x[idx] = re.sub("_","...",x[idx])
    
    return x

# fungsi untuk menyimpan hasil ke bentuk list
def cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan):
    x = [lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan]
    
    x = replaceData(x)
    
    #split panah
    if x[4] != "" :
        x[4] = re.split(",|;",x[4])
    
    #split terjemahan
    if x[9] != "" :
        x[9] = re.split(",|;",x[9])
        x[9] = replaceData(x[9])
        
        if "" in x[9] :
            x[9].remove("")
        
        for idx in range(len(x[9])):
            x[9][idx] = re.sub("_","...",x[9][idx])
            
        if len(x[9]) < 1 :
            x[9] = ""
    
    y = pd.DataFrame(data=[x],columns=["lema","sublema","gabungan","similar","panah","akronim","antonim","perbandingan","keterangan","terjemahan"])
    
    return x,y

def cleanParenthesis(word) :
    if '(' in word :
        x, y = re.sub(r"[/(].*?[/))=]","",word),re.sub(r"[()=]","",word)
        x, y = re.sub(r"\s$","",x),re.sub(r"\s$","",y)
        word = x
        similar = y
        tempLemaSublema = word
        boolLemaSublema = True
    else :
        word = re.sub(r"[()]|\s$","",word)
        similar = ""
        tempLemaSublema = word
        boolLemaSublema = True
        
    return word,similar,tempLemaSublema,boolLemaSublema

def readItalicData(word,index,paragraph,keterangan,tempKeterangan,tempGabungan,terjemahan) :     
    if word == "opp" or word == "cp":
        word = ""

    #KETERANGAN
    # mengambil keterangan
    if index > 0 and re.search("\(", paragraph.runs[index-1].text):

        if re.search("\s", word) :
            split_text = word.split()
            tempWord = []

            for s in split_text :
                if s in list_ket :
                    tempWord.append(s)
                else :
                    break

            word = " ".join(tempWord)

            tempKeterangan.append(word)

        elif word in list_ket :
            tempKeterangan.append(word)
            
        else :
            #update 2
            
#             try :
            if (index+1 < len(paragraph.runs) and paragraph.runs[index+1].italic) or (index+1 < len(paragraph.runs) and re.search("\)", paragraph.runs[index+1].text)) :
                tempGabungan.append(word)
            else :
                if terjemahan == "" and word not in roman_numerals and boolPanah != True:
                    terjemahan = word
                elif terjemahan != "" and word not in roman_numerals and boolPanah != True:
                    terjemahan = terjemahan+" "+word
#             except :
#                 if terjemahan == "" and word not in roman_numerals and boolPanah != True:
#                     terjemahan = word
#                 elif terjemahan != "" and word not in roman_numerals and boolPanah != True:
#                     terjemahan = terjemahan+" "+word

        # update sementara
        try :
            if re.search("\)", word) or (index+1 < len(paragraph.runs) and re.search("\)", paragraph.runs[index+1].text)) :
                keterangan = " ".join(tempKeterangan)
                tempKeterangan = []
        except :
            if re.search("\)", word) :
                keterangan = " ".join(tempKeterangan)
                tempKeterangan = []
        
#         keterangan = "satu"

    # mengambil keterangan selanjutnya jika terdiri dari > 1 keterangan
    elif len(tempKeterangan) > 0 and word not in tempKeterangan:
        if re.search("\s", word) :
            split_text = word.split()
            tempWord = []

            for s in split_text :
                if s in list_ket :
                    tempWord.append(s)
                else :
                    break

            word = " ".join(tempWord)

            tempKeterangan.append(word)

        elif word in list_ket :
            tempKeterangan.append(word)

            try :
                if (index+1 < len(paragraph.runs) and re.search("\)", paragraph.runs[index+1].text)) or re.search("\)", word) :
                    keterangan = " ".join(tempKeterangan)
                    tempKeterangan = []
            except :
                print("terjadi kesalahan")
                
        else :
            if terjemahan == "" and word not in roman_numerals and boolPanah != True:
                terjemahan = word
            elif terjemahan != "" and word not in roman_numerals and boolPanah != True:
                terjemahan = terjemahan+" "+word
#         keterangan = "dua"

    else :
        tempGabungan.append(word)
#         keterangan = "tiga"
    
    return keterangan,tempKeterangan,tempGabungan,terjemahan

In [7]:
lexicon = pd.DataFrame(columns=["lema","sublema","gabungan","similar","panah","akronim","antonim","perbandingan","keterangan","terjemahan"])
roman_numerals = ["I","II","III","IV","V","VI","VII","VIII","IX","X"]
list_and = ["and", " and", "and ", " and ", "[and", "[and "]
list_ket = ["A","abbr","Ac","acr","adj","AE","anat","app","Bal","Ban","Bat","BD","BE","BG","bio","bot","C","chem","cla","col","coq","cp","D","D/E","derog","E","elec","e.o.","epist","esp","euph","exclam","fin","Fr","G","geo","geol","Gr","gram","Hind","IBT","infr","init","insur","Irja","Isl","J","J/Jv","joc","Jp","Jv","K","k.o.","L","leg","ling","lit","M","Mad","Mal","math","Med","med","mil","Min","mod","mus","naut","NTB","NTT","ob","obj","O jv","onom","opp","o.s.","Pal","Pap","Pers","petro","phys","pl","pl obj","pl subj","poet","Port","Pr","pron","q.v.","RC","reg","rel","rev","S","sg","Sg","Skr","Skr neo","sl","s.o.","s.o.'s","s.t.","stat","subj","Sum","Tag","Tam","usu","voc","vulg","zod"]

#variabel
lema = ""
sublema = ""
gabungan = ""
similar = ""
panah = ""
akronim = ""
antonim = ""
perbandingan = ""
keterangan = ""
terjemahan = ""

boolLemaSublema = False
boolPanah = False
tempLema = ""
tempSublema = ""
# tempLemaSublema = ""
tempTerjemahan = ""
tempGabungan = []
tempKeterangan = []
tempAkronim = []
    
index = 0

#looping paragraf tiap document
for paragraph in document.paragraphs:
    
    boolLemaSublema = False
    
    index = 0
    
    #menyimpan nilai indentasi (first indent dan left indent)
    first = paragraph.paragraph_format.first_line_indent
    left = paragraph.paragraph_format.left_indent
    
    #looping setiap kata di setiap paragrafnya
    for run in paragraph.runs:
        
#         print("word = "+run.text)
#         print("index = "+str(index))
#         print("length = "+str(len(paragraph.runs)))
        
        #mengatasi left indent == none, dengan mengubah nilai variabel left dengan tempLeft (nilai pada left sebelumnya)
        if left == None :
            left = 228600 
        
        #deteksi kata bold
        if run.bold and run.text.isspace() == False :
            if run.text.isspace() == False :
                #baca indent lema
                if (((first == None or first == 0 or first == 76200) and (left <= 76835)) or ((first == -152400 or first == -153035) and left >= 227965)) :      

                    tempLeft = left

                    if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                        x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)
                        print(x[0])
                        lexicon  = lexicon.append([x[1]],ignore_index=True)

                    #variabel 
                    lema = ""
                    sublema = ""
                    gabungan = ""
                    similar = ""
                    panah = ""
                    akronim = ""
                    antonim = ""
                    perbandingan = ""
                    keterangan = ""
                    terjemahan = ""

                    boolPanah = False
                    tempGabungan = []
                    tempKeterangan = []
                    tempAkronim = []

                    word = cleanWord(run.text)

                    if word != None :
                        
#                         print("run.text = "+run.text)
#                         print(word)
                        
                        #LEMA
                        #jika lema kosong atau belum terdapat lema, lema = word
                        if boolLemaSublema == False :

                            results = cleanParenthesis(word)

                            lema = results[0]
                            similar = results[1]
#                             tempLemaSublema = results[2]
                            boolLemaSublema = [3]
                            
                            tempLema = lema

                        #jika lema tidak kosong atau variabel lema sudah terisi
                        else :
#                             print("prev = "+paragraph.runs[index-1].text)
#                             print("word = "+word)
                            
                            #SIMILAR (AND)
                            #jika kata sebelum kata style bold adalah "and" maka dianggap sebagai similar
                            if (index > 0 and paragraph.runs[index-1].text in list_and) or (index > 1 and paragraph.runs[index-2].text in list_and) :
#                                 print(word)
                                similar = word

                            #jika bukan similar
                            else :  
                                #SUBLEMA(JIKA SEBARIS/SEPARAGRAF DENGAN LEMA)  
                                #jika sublema kosong
                                if sublema == "" :
                                    if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                                        x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)
                                        print(x[0])
                                        lexicon  = lexicon.append([x[1]],ignore_index=True)

                                    lema = ""
                                    similar = ""
                                    gabungan = ""
                                    similar = ""
                                    panah = ""
                                    akronim = ""
                                    antonim = ""
                                    perbandingan = ""
                                    keterangan = ""
                                    terjemahan = ""

                                    results = cleanParenthesis(word)

                                    sublema = results[0]
                                    similar = results[1]
#                                     tempLemaSublema = results[2]
                                    tempSublema = results[2]
                                    boolLemaSublema = [3]

                                #jika sublema tidak kosong, maka simpan sublema ke list terlebih dahulu
                                else :
                                    if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                                        x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)
                                        print(x[0])
                                        lexicon  = lexicon.append([x[1]],ignore_index=True)

                                    lema = ""
                                    similar = ""
                                    gabungan = ""
                                    similar = ""
                                    panah = ""
                                    akronim = ""
                                    antonim = ""
                                    perbandingan = ""
                                    keterangan = ""
                                    terjemahan = ""

                                    results = cleanParenthesis(word)

                                    sublema = results[0]
                                    similar = results[1]
#                                     tempLemaSublema = results[2]
                                    tempSublema = results[2]
                                    boolLemaSublema = [3]

                #baca indent sublema
                else :
                    tempLeft = left 

                    #deteksi kata bold
                    if(run.bold) and run.text.isspace() == False :

                        if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                            x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)
                            print(x[0])
                            lexicon  = lexicon.append([x[1]],ignore_index=True)

                        #variabel
                        lema = ""
                        sublema = ""
                        gabungan = ""
                        similar = ""
                        panah = ""
                        akronim = ""
                        antonim = ""
                        perbandingan = ""
                        keterangan = ""
                        terjemahan = ""

                        boolPanah = False
                        tempGabungan = []
                        tempKeterangan = []
                        tempAkronim = []

                        word = cleanWord(run.text)

                        if word != None :
                            
#                             print("prev = "+paragraph.runs[index-1].text)
#                             print("word = "+word)
                            
                            #SUBLEMA
                            #jika sublema kosong
                            if boolLemaSublema == False :

                                results = cleanParenthesis(word)

                                sublema = results[0]
                                similar = results[1]
#                                 tempLemaSublema = results[2]
                                tempSublema = results[2]
                                boolLemaSublema = [3]

                            #jika sublema tidak kosong
                            else :
                                #SIMILAR (AND)
                                #jika kata sebelum kata style bold adalah "and" maka dianggap sebagai similar
                                if (index > 0 and paragraph.runs[index-1].text in list_and) or (index > 1 and paragraph.runs[index-2].text in list_and) :
                                    similar = word

                                #SUBLEMA (JIKA SUBLEMA SEBARIS/SEPARAGRAF DENGAN SUBLEMA YANG LAIN)
                                #jika bukan, maka simpan sublema ke list terlebih dahulu
                                else :
                                    if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                                        x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)
                                        print(x[0])
                                        lexicon  = lexicon.append([x[1]],ignore_index=True)

                                    lema = ""
                                    sublema = ""
                                    gabungan = ""
                                    similar = ""
                                    panah = ""
                                    akronim = ""
                                    antonim = ""
                                    perbandingan = ""
                                    keterangan = ""
                                    terjemahan = ""

                                    results = cleanParenthesis(word)

                                    sublema = results[0]
                                    similar = results[1]
#                                     tempLemaSublema = results[2]
                                    tempSublema = results[2]
                                    boolLemaSublema = [3]

        #deteksi kata italic (kata gabungan, keterangan)
#             print("sublema = "+sublema)
#             print("similar = "+similar)
#             print("tempLemaSublema = "+tempLemaSublema)
            
        elif run.italic :
            if run.text.isspace() == False :
                results = readItalicData(run.text,index,paragraph,keterangan,tempKeterangan,tempGabungan,terjemahan)

#                 print("prev = "+paragraph.runs[index-1].text)
#                 print(paragraph.runs[index-1].italic)
#                 print("text italic = "+run.text)
                
                keterangan = results[0]
                tempKeterangan = results[1]
                tempGabungan = results[2] 
                terjemahan = results[3]
                
#                 print(results)
                
        # deteksi kata reguler (akronim, terjemahan)
        else : 
            if run.text.isspace() == False :
                word = run.text

                if word != None :
                    #GABUNGAN
#                     if "–" in tempGabungan or "~" in tempGabungan:
#                         gabungan = " ".join(tempGabungan)
#                         gabungan = re.sub("–|~",tempLemaSublema,gabungan)
#                         gabungan = re.sub("  "," ", gabungan)

#                         results = cleanParenthesis(gabungan)

#                         gabungan = results[0]
#                         similar = results[1]

#                         tempGabungan = []

#                     print(tempGabungan)

                    if "–" in tempGabungan:
                        gabungan = " ".join(tempGabungan)
                        gabungan = re.sub("–",tempLema,gabungan)
                        gabungan = re.sub("  "," ", gabungan)

                        results = cleanParenthesis(gabungan)
                        
                        gabungan = results[0]
                        similar = results[1]
                        
                        if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                            x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)

                            print(x[0])
                            lexicon  = lexicon.append([x[1]],ignore_index=True)

                        lema = ""
                        sublema = ""
                        gabungan = ""
                        similar = ""
                        panah = ""
                        akronim = ""
                        antonim = ""
                        perbandingan = ""
                        keterangan = ""
                        terjemahan = ""
                            
                        boolPanah = False
                        tempGabungan = []
                        tempKeterangan = []
                        tempAkronim = []

#                         gabungan = results[0]
#                         similar = results[1]

#                         tempGabungan = []
                    
                    elif "~" in tempGabungan:
                        gabungan = " ".join(tempGabungan)
                        gabungan = re.sub("~",tempSublema,gabungan)
                        gabungan = re.sub("  "," ", gabungan)

                        results = cleanParenthesis(gabungan)
                        
                        gabungan = results[0]
                        similar = results[1]
                        
                        if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                            x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)

                            print(x[0])
                            lexicon  = lexicon.append([x[1]],ignore_index=True)

                        lema = ""
                        sublema = ""
                        gabungan = ""
                        similar = ""
                        panah = ""
                        akronim = ""
                        antonim = ""
                        perbandingan = ""
                        keterangan = ""
                        terjemahan = ""
                            
                        boolPanah = False
                        tempGabungan = []
                        tempKeterangan = []
                        tempAkronim = []

#                         gabungan = results[0]
#                         similar = results[1]

#                         tempGabungan = []

                    else :
                        if len(tempGabungan) != 0:
                            
#                             print ("length = "+str(len(tempGabungan)))
#                             print ("index = "+str(index))
#                             print(tempGabungan)
#                             print("tempLema = "+tempLema )

                            temp = " ".join(tempGabungan)
    
#                             print("temp = "+temp)
                            
                            if boolPanah == True and panah != word and word not in roman_numerals :
#                                 temp = " ".join(tempGabungan)
                                panah = panah+" "+temp

                                temp = ""
                                tempGabungan = []
                            
#                             elif len(tempGabungan) > 1 and (tempLema.lower() in temp.lower() or tempLemaSublema.lower() in temp.lower()) :
                            elif len(tempGabungan) > 1 and (tempLema.lower() in temp.lower() or tempSublema.lower() in temp.lower()) :
#                                 temp = " ".join(tempGabungan)
                                gabungan = gabungan+" "+temp
                                gabungan = re.sub(r'  ',' ', gabungan)

                                results = cleanParenthesis(gabungan)

                                gabungan = results[0]
                                similar = results[1]

                                tempGabungan = []
                            
                            elif index > 1 and len(tempGabungan) <= 3:
#                                 temp = " ".join(tempGabungan)
                                terjemahan = terjemahan+" "+temp

                                temp = ""
                                tempGabungan = []

                            else :
#                                 temp = " ".join(tempGabungan)
                                gabungan = gabungan+" "+temp
                                gabungan = re.sub(r'  ',' ', gabungan)

                                results = cleanParenthesis(gabungan)

                                gabungan = results[0]
                                similar = results[1]

                                tempGabungan = []

                #AKRONIM
                # mengambil akronim
                if (index > 0 and re.search("\[", paragraph.runs[index-1].text)) and word not in list_and:
                    tempAkronim.append(word)

                    if (index+1 < len(paragraph.runs) and re.search("\]", paragraph.runs[index+1].text)) :
                        akronim = " ".join(tempAkronim)
                        tempAkronim = []

                # mengambil akronim selanjutnya jika terdiri > 1 kata
                elif len(tempAkronim) > 0 and word not in tempAkronim :
                    tempAkronim.append(word)

#                     try :
                    if (index+1 < len(paragraph.runs) and re.search("\]", paragraph.runs[index+1].text)) :
                        akronim = " ".join(tempAkronim)
                        akronim = re.sub(r'  ',' ', akronim)
                        tempAkronim = []
#                     except :
#                         if re.search("\]", paragraph.runs[index].text) :
#                             akronim = " ".join(tempAkronim)
#                             akronim = re.sub(r'  ',' ', akronim)
#                             tempAkronim = []
                else :
                    #PANAH   
                    
#                     print("prev-1 = "+paragraph.runs[index-1].text)
                    
#                     if index > 1 :
#                         print("prev-2 = "+paragraph.runs[index-2].text)
                        
#                     print("word = "+word)
#                     print("terjemahan = "+terjemahan)
#                     print("index = "+str(index))
#                     print("length = "+str(len(paragraph.runs)))
                    
                    if boolPanah == True and word not in roman_numerals :
                        tempWord = re.sub("\.","", word)
                        tempWord = cleanWord(tempWord)
                     
#                         print("word 2= "+word)
#                         print(tempWord)

                        if tempWord != None :
                            panah = panah+" "+tempWord
                            word = ""
                            
#                         print("panah 2 = "+panah)
                    
                    if boolPanah == True and re.search("\.", word) :
#                         print("word 3= "+word)
                        
                        panah = re.sub("\.|\s\.","",panah)
                        boolPanah = False
                        word = ""
#                         print("panah 3 = "+panah)
                    
                    
#                     if word != "" and word != "." and (paragraph.runs[index-1].text == ">" or (index > 1 and paragraph.runs[index-2].text == ">")) :
                    if word != "" and word != "." and ((index > 0 and re.search(">", paragraph.runs[index-1].text)) or (index > 1 and re.search(">", paragraph.runs[index-2].text))) :

#                         print("run.text = "+run.text)
#                         print("word 1= "+word)

                        word = re.sub("\.","", word)
                        word = cleanWord(word)

                        if word != None :
                            panah = word
                            boolPanah = True
                            word = ""
                            
#                         print("panah = "+panah)
                        
#                         print("panah 1 = "+panah)
#                             
#                             try :
#                                 print(paragraph.runs[index+1].text)
#                                 print(panah)
#                             except :
#                                 print(paragraph.runs[index+0].text)
#                                 print(panah)

                    elif (index > 0 and paragraph.runs[index-1].italic == True) and ((index > 0 and paragraph.runs[index-1].text == "opp") or (index > 1 and paragraph.runs[index-2].text == "opp")):
                        re.sub("\s","",word)

                        antonim = word
                
                    elif (index > 0 and paragraph.runs[index-1].italic == True) and ((index > 0 and paragraph.runs[index-1].text == "cp") or (index > 1 and paragraph.runs[index-2].text == "cp")):
                        re.sub("\s","",word)

                        perbandingan = word

                    else :

                        #TERJEMAHAN
                        if re.search("\]\s", word) :
                            word = re.sub("\]\s","",word)

                        if word in list_and and ((index > 0 and paragraph.runs[index-1].bold) == True and ((index+1 < len(paragraph.runs) and paragraph.runs[index+1].bold == True) or (index+2 < len(paragraph.runs) and paragraph.runs[index+2].bold == True))):
                            word = ""

                        if word == "[" or word == "]" or re.search(">", word) :
                            word = ""

#                         if terjemahan == "" and word not in roman_numerals and boolPanah != True :
                        if terjemahan == "" and boolPanah != True :
                            terjemahan = word
#                         elif terjemahan != "" and word not in roman_numerals and boolPanah != True:
                        elif terjemahan != "" and boolPanah != True:
                            terjemahan = terjemahan+" "+word

#                         print("prev = "+paragraph.runs[index-1].text)
#                         print("word = "+word)
#                         print("terjemahan = "+terjemahan)
#                         print("run.text = "+run.text)
                        if re.search("\.|\;|\!|\?", run.text) or re.match("\w\)", run.text) :
        
#                             print("hihi")
            
                            if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                                x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)

                                print(x[0])
                                lexicon  = lexicon.append([x[1]],ignore_index=True)

                            lema = ""
                            sublema = ""
                            gabungan = ""
                            similar = ""
                            panah = ""
                            akronim = ""
                            antonim = ""
                            perbandingan = ""
                            keterangan = ""
                            terjemahan = ""
                            
                            boolPanah = False
                            tempGabungan = []
                            tempKeterangan = []
                            tempAkronim = []
                            
#                             boolPanah = False
        index += 1                 

['cokin', '', '', '', '', '', '', '', 'Pr', ['(a slur for) Indonesianborn Chinese']]
['', '', '', '', ['CINA'], '', '', '', '', '']
['coklat', '', '', '', '', '', '', '', 'D', '']
['', '', 'pohon coklat', '', '', '', '', '', '', '']
['', '', 'Theobroma cacao', '', '', '', '', '', '', ['cacao tree', 'whose seeds are made into a powder']]
['', '', '', '', '', '', '', '', '', ['chocolate', 'a drink made by adding sugar and milk/hot water to the above powder']]
['', '', '', '', '', '', '', '', '', ['chocolate', 'sweet substance made with cocoa powder', 'sugar', 'etc']]
['', '', '', '', '', '', '', '', '', ['chocolate', 'a reddishyellow brown color']]
['', '', 'Gadis itu mengenakan seragam – dengan pét putih', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['The girl was wearing a reddishyellow uniform with white cap']]
['', '', '', '', '', '', '', '', '', ['– betas sugared caraway seeds (to be sprinkled on bread)']]
['', '', '', '', '', '', '', '', '', ['– sampang rosewoo

['', '', '', 'nyoliin', '', '', '', '', 'J coq', ['to masturbate someone', 'rub (a part of the body) in a sexual way']]
['colok', '', '', '', '', '', '', '', '', '']
['', '', 'lampu colok', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['torch', 'made of a piece of twisted cloth dipped in oi l (use d a s a light)']]
['', '', '', '', '', '', '', '', '', ['a p a l ml e a f r i b p r o v i d e d w i t h r e s i n /d a m a r or sulfur to function as a light or to replace a match', 'etc']]
['', '', '', '', '', '', '', '', '', ['white stripes on the tail of a black dog']]
['', 'menyolok', '', '', '', '', '', '', '', '']
['', '', '', 'menyolok/nyolok', '', '', '', '', 'coq', ['to light up', 'illuminate']]
['', 'mencoloki', '', '', '', '', '', '', '', ['to illuminate with a torch', 'etc']]
['colok', '', '', '', '', '', '', '', '', '']
['', '', '', '', ['CELUP'], '', '', '', 'M', '']
['', '', '', '', '', '', '', '', '', ['jatuh/turun –nya in reduced circumstances', 'went down

['', '', '', '', '', '', '', '', '', ['that acted as a middleman in this meeting as part of its strategy to arrange a “new world order.”']]
['', 'percomblangan', '', '', '', '', '', '', 'mod', ['matchmaker']]
['combol', '', '', '', '', '', '', '', 'ob', ['doorknob']]
['', '', '', '', ['TOMBOL'], '', '', '', '', '']
['combong', '', '', '', '', '', '', '', 'J', ['a small kris (with a perforation in the blade as an amulet)']]
['combor', '', '', '', '', '', '', '', 'Jv', ['horse feed: grass', 'rice bran', 'and rice husks mixed with water']]
['combro', '', '', 'combrok', '', '', '', '', 'J', ['kind(s) of food made of cassava filled with oncom']]
['', '', 'tukang combro', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['person who produces or sells this food']]
['comék', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', 'J', ['hairs that grow between the chin and lower lip (an imperial beard)']]
['comék', '', '', '', '', '', '', '', '', ['a small cuttlefi

['', '', '', '', '', '', '', '', '', ['to incline', 'tend']]
['', '', '', '', '', '', '', '', '', ['~ kepada to favor']]
['', '', '', '', '', '', '', '', '', ['to show affection for']]
['', 'tercondong', '', '', '', '', '', '', '', ['inclined']]
['', '', '', '', '', '', '', '', '', ['~ balik reclined']]
['', 'kecondongan', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['tendency', 'penchant']]
['', '', '', '', '', '', '', '', '', ['inclination', 'leaning']]
['', '', '', '', '', '', '', '', '', ['bias']]
['condrong', '', '', '', ['CENDERUNG'], '', '', '', '', '']
['Conéfo /', '', '', '', '', 'Conference of the New Emerging Forces', '', '', '', ['ko//ko/President Soekarno’s concept of a rival organization to the UN']]
['conét menconét', '', '', '', '', '', '', '', '', ['pointed and upturning (as a mustache)']]
['cong', '', '', '', '', '', '', '', '', ['(Madura and East Java) vocative for a child']]
['congak bercongakan', '', '', '', '', '', '', '', 'pl subj', ['to 

['', 'conténgan', '', '', '', '', '', '', '', ['scrawl', 'scribble']]
['conto', '', '', '', ['CONTOH'], '', '', '', '', '']
['contoh', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['sample', 'specimen']]
['', '', 'tidak cocok dengan –nya', '', '', '', '', '', '', ['not corresponding to the sample']]
['', '', '', '', '', '', '', '', '', ['model']]
['', '', 'Guru membuat contoh di papan tulis', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['The teacher made a model on the blackboard']]
['', '', '', '', '', '', '', '', '', ['pattern']]
['', '', 'Dalam ma-jalah wanita selalu dimuat contoh', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['.–']]
['', '', '', '', '', '', '', '', '', ['menggunting baju']]
['', '', '', '', '', '', '', '', '', ['Women’s magazines always carry dressmaking patterns']]
['', '', '', '', '', '', '', '', '', ['mockup', 'model']]
['', '', '', '', '', '', '', '', 

...

['', '', '', '', '', '', '', '', 'ob', ['sovereign', 'monarch', 'king']]
['', '', '', '', ['DATUK ', ' RATU'], '', '', '', '', '']
['', '', '', '', '', '', '', '', 'M', ['medicine man', 'shaman']]
['', '', '', '', ['BOMOH ', ' DUKUN'], '', '', '', '', '']
['', '', '', '', '', '', '', '', 'J', ['a holy person', 'saint']]
['', '', '', '', '', '', '', '', '', ['a deceased person']]
['', 'kedatua', '', '', '', '', '', '', '', '']
['', 'n', '', '', '', '', '', '', 'ob', '']
['', '', '', '', '', '', '', '', '', ['kingdom']]
['', '', '', '', '', '', '', '', '', ['~ Sriwijay a t h e S r i w i j a y a K i n g d o m (of Sumatra)']]
['datu kedatuan', '', '', '', '', '', '', '', 'M', ['kind(s) of dance']]
['datuk', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['head', 'elder']]
['', '', '', '', '', '', '', '', '', ['grandfather']]
['', '', '', '', '', '', '', '', 'M', ['an honorific title', 'i.e', 'a gelar for people of high status or older people within the group']]
['

['', '', '', '', '', '', '', '', '', ['– jendéla window shutter']]
['', '', '', '', '', '', '', '', '', ['– jeruk purut citrus leaf']]
['', '', '', '', '', '', '', '', '', ['– jingga orang leaf']]
['', '', '', '', '', '', '', '', '', ['– jinten kind(s) of plant similar to oregano', 'Coleus amboinicus']]
['', '', '', '', '', '', '', '', '', ['– kaki kuda a medicinal plant', 'Indian pennywort', 'Centella asiatica']]
['', '', '', '', '', '', '', '', '', ['– katuk sweetleaf bush', 'Sauropus androgynus']]
['', '', '', '', '', '', '', '', '', ['– keji beling Persian shield', 'Strobilanthes spp']]
['', '', '', '', '', '', '', '', 'bio', ['– kelopak sepal']]
['', '', '', '', '', '', '', '', '', ['– kemudi rudder']]
['', '', '', '', '', '', '', '', '', ['– kentut skunk vine', 'Paederia scandens']]
['', '', '', '', '', '', '', '', '', ['– kepialu kind(s) of medicinal plant', 'Vitis trifolia']]
['', '', '', '', '', '', '', '', '', ['– kemarogan kind(s) of plant', 'Gymnopetalum leucostictum']]
[''

['', 'mendaur-ulang', '', 'mendaur-ulangkan', '', '', '', '', '', ['to recycle']]
['', '', 'dapat didaurulang', '', '', '', '', '', '', ['recyclable']]
['', 'pendaur-ulang', '', '', '', '', '', '', '', ['recycling (machine/plant)']]
['', 'pendaur-ulangan', '', '', '', '', '', '', '', '']
['', '', '', 'pendauran-ulang', '', '', '', '', 'infr', ['recycling']]
['', 'mendaurkan', '', '', '', '', '', '', '', ['to (re)cycle']]
['', 'terdaurkan', '', '', '', '', '', '', '', ['(re)cycled']]
['', '', 'sampah', '', '', '', '', '', '', ['~ recycled garbage/trash/waste']]
['dauriah', '', '', '', '', '', '', '', 'A', ['cyclical']]
['da’wa', '', '', '', ['DAKWA'], '', '', '', '', '']
['da’wah', '', '', '', ['DAKWAH'], '', '', '', '', '']
['dawai', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['string', 'cord']]
['', '', '', '', '', '', '', '', '', ['metallic wire']]
['', '', '', '', ['KAWAT'], '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['– duri barbed wire']

['', '', '', '', '', '', '', '', '', ['– juang fighting spirit']]
['', '', '', '', '', '', '', '', '', ['– kandung air waterholding capacity']]
['', '', '', '', '', '', '', '', '', ['– karya work capacity']]
['', '', '', '', '', '', '', '', '', ['– kata catchword', 'motto']]
['', '', '', '', '', '', '', '', '', ['– kebal immunity', 'resistance']]
['', '', '', '', '', '', '', '', '', ['– kembali ability to spring back', 'resilience']]
['', '', '', '', '', '', '', '', '', ['– kecambah germinative power']]
['', '', '', '', '', '', '', '', '', ['– kejut shock action']]
['', '', '', '', '', '', '', '', '', ['– kenyal resilience', 'resiliency', 'spring']]
['', '', '', '', '', '', '', '', '', ['– kerja']]
['', '', '', '', '', '', '', '', '', ['work capacity']]
['', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['energy']]
['', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['ability to work']]
['', '', '', '', '', '', '', '', '', ['– kerja 

['', 'g', '', '', '', '', '', '', '', ['w i t h e l e c t r i c a l service (of ... volts)']]
['', '', '', '', '', '', '', '', '', ['– tersedia available power']]
['', '', '', '', '', '', '', '', '', ['– tetas hatchability']]
['', '', 'timban g', '', '', '', '', '', '', ['– judgment']]
['', '', 'tola k', 'tola k balik', '', '', '', '', '', ['– recoil']]
['', '', '', '', '', '', '', '', '', ['– tua s l e v e r a g e']]
['', '', '', '', '', '', '', '', '', ['– tumbuh viability']]
['', '', '', '', '', '', '', '', '', ['– ujud enforcement']]
['', '', '', '', '', '', '', '', '', ['– ungkapan expressiveness']]
['', '', '', '', '', '', '', '', '', ['– ungkit leverage']]
['', '', '', '', '', '', '', '', '', ['– upaya efforts']]
['', 'sedaya-upaya', '', '', '', '', '', '', '', ['to the best of one’s ability', 'utmost']]
['', 'berdaya-upaya', '', '', '', '', '', '', '', ['to work hard at', 'try hard to']]
['', 'mendaya-upayakan', '', '', '', '', '', '', '', ['to do one’s best to', 'make every ef

In [8]:
lexicon

,lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan
0,cokin,,,,,,,,Pr,[(a slur for) Indonesianborn Chinese]
1,,,,,[CINA],,,,,
2,coklat,,,,,,,,D,
3,,,pohon coklat,,,,,,,
4,,,Theobroma cacao,,,,,,,"[cacao tree, whose seeds are made into a powder]"
...,...,...,...,...,...,...,...,...,...,...
5494,,dbp,,,,di bawah pimpinan,,,abbr,[directed by]
5495,,DD,,,,,,,,[car license for South Sulawesi]
5496,,DE,,,,,,,,[car license for South Maluku]
5497,,dé,,,,,,,,


In [9]:
# lexicon.to_excel (r'A.xlsx', index = False, header=True)